In [31]:
import numpy as np
import random
import math
import time

In [32]:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

print("VALUES:\n", VALUES)
print("\nWEIGHTS:\n", WEIGHTS)
print("\nCONSTRAINTS:\n", CONSTRAINTS)

VALUES:
 [ 8 77 65 43 43 85  8 69 20  9 52 97 73 76 71 78 51 12 83 45]

WEIGHTS:
 [[50 37]
 [18 92]
 [78 64]
 [40 82]
 [54 44]
 [45 22]
 [ 9 55]
 [88  6]
 [85 82]
 [27 63]
 [16 75]
 [70 35]
 [ 6 97]
 [44 89]
 [67 77]
 [75 19]
 [36 46]
 [49  4]
 [54 15]
 [74 68]]

CONSTRAINTS:
 [[614 496]
 [244 644]
 [273 216]]


In [ ]:
def evaluate(solution, values, weights, constraints):

    num_knapsacks, num_dims = constraints.shape
    loads = np.zeros((num_knapsacks, num_dims), dtype=int)
    total_value = 0

    for i, k in enumerate(solution):
        if k > 0:  # item assigned to knapsack k
            loads[k - 1] += weights[i]
            total_value += values[i]

    feasible = np.all(loads <= constraints)
    violation = np.maximum(0, loads - constraints).sum()

    return total_value, loads, feasible, violation

In [ ]:
def greedy_initial(values, weights, constraints):

    num_items = len(values)
    num_knapsacks, num_dims = constraints.shape
    order = sorted(range(num_items),
                   key=lambda i: values[i] / (1 + np.sum(weights[i])),
                   reverse=True)
    loads = np.zeros((num_knapsacks, num_dims), dtype=int)
    sol = [0] * num_items

    for i in order:
        for k in range(num_knapsacks):
            if np.all(loads[k] + weights[i] <= constraints[k]):
                sol[i] = k + 1
                loads[k] += weights[i]
                break
    return sol

In [ ]:
def simulated_annealing(values, weights, constraints,
                        max_iters=10000,
                        cooling_rate=0.995,
                        penalty_lambda=50.0,
                        iterations_per_temp=300):

    # Initial feasible solution
    current_sol = greedy_initial(values, weights, constraints)
    current_val, _, current_feas, current_viol = evaluate(current_sol, values, weights, constraints)
    current_energy = -current_val + penalty_lambda * current_viol

    best_sol = current_sol.copy()
    best_val = current_val
    best_feas_sol = current_sol.copy() if current_feas else None
    best_feas_val = current_val if current_feas else -1

    # Estimate initial temperature
    sample_energies = []
    for _ in range(30):
        i = random.randrange(len(values))
        new_k = random.randrange(0, constraints.shape[0] + 1)
        cand = current_sol.copy()
        cand[i] = new_k
        val, _, _, viol = evaluate(cand, values, weights, constraints)
        e = -val + penalty_lambda * viol
        sample_energies.append(abs(e - current_energy))
    T = np.mean(sample_energies) / math.log(2) if np.mean(sample_energies) > 0 else 1.0
    Tmin = 1e-4

    start_time = time.time()
    total_iter = 0

    # MAIN LOOP
    while T > Tmin and total_iter < max_iters:
        for _ in range(iterations_per_temp):
            total_iter += 1

            # Generate neighbor
            i = random.randrange(len(values))      # pick random item
            new_k = random.randrange(0, constraints.shape[0] + 1)  # assign new knapsack (or 0)
            candidate = current_sol.copy()
            candidate[i] = new_k

            cand_val, _, cand_feas, cand_viol = evaluate(candidate, values, weights, constraints)
            cand_energy = -cand_val + penalty_lambda * cand_viol
            deltaE = cand_energy - current_energy

            # Acceptance criterion
            if deltaE < 0 or random.random() < math.exp(-deltaE / T):
                current_sol, current_val, current_energy = candidate, cand_val, cand_energy
                if cand_feas and cand_val > best_feas_val:
                    best_feas_sol = candidate.copy()
                    best_feas_val = cand_val
                if cand_val > best_val:
                    best_val = cand_val
                    best_sol = candidate.copy()

        # Cool down temperature
        T *= cooling_rate

    elapsed = time.time() - start_time

    return {
        "best_feasible_value": best_feas_val,
        "best_feasible_sol": best_feas_sol,
        "best_overall_value": best_val,
        "best_overall_sol": best_sol,
        "time": elapsed
    }

In [36]:
result = simulated_annealing(VALUES, WEIGHTS, CONSTRAINTS)

print("\n--- SIMULATED ANNEALING RESULT ---")
print("Best feasible value:", result["best_feasible_value"])
print("Best feasible assignment:", result["best_feasible_sol"])
val, loads, feas, viol = evaluate(result["best_feasible_sol"], VALUES, WEIGHTS, CONSTRAINTS)
print("Feasible:", feas, "Violation:", viol)
print("Knapsack loads:\n", loads)
print("Runtime (seconds):", result["time"])


--- SIMULATED ANNEALING RESULT ---
Best feasible value: 1048
Best feasible assignment: [0, 1, 2, 2, 2, 1, 3, 1, 3, 0, 1, 1, 1, 1, 2, 1, 1, 3, 1, 3]
Feasible: True Violation: 0
Knapsack loads:
 [[452 496]
 [239 267]
 [217 209]]
Runtime (seconds): 0.2765989303588867
